# Estadística Aplicada

## Actividad 6 parte A

###Paulina Quiroz Sánchez 1723927 Grupo 41

Cada vez que un empleado renuncia a una empresa “X”, a esta empresa le toma como mínimo  6  meses  contratar  un  remplazo,  más  3  meses  de  capacitación  y  un  mes  de adaptación.  Tal  situación  implica pérdida de tiempo  y recursos  por  los  siguientes motivos:

a)Cuando  un  empleado  se  va,  sus  responsabilidades  deben  ser  redistribuidas entre  los  empleados  restantes.  Esto  puede  generar  una  carga  de  trabajo adicional para aquellos que quedan, lo que puede llevar a la fatiga

b)Reemplazar a un empleado puede ser costoso. Incluye gastos en reclutamiento, selección, capacitación y adaptación

c)La rotación frecuente de empleados puede llevar a interrupciones en el flujo de trabajo,  ya  que  los  nuevos  empleados  necesitan  tiempo  para  adaptarse  y alcanzar el mismo nivel de productividad que sus predecesores.

Por tal razón la empresa “X” desea conocer las razones que hacen que sus empleados renuncien,    para    tal    fin    le    ha    proporcionado una    base    de    datos llamada deserción_laboral que mide el tiempo en semestres desde que un empleado entró a trabajara la empresa “X” hasta que renuncia. Donde el evento es renunciar.

Revisar tabla.

Dada la información proporcionada el modelo más adecuado es el modelo de Cox de riesgos proporcionales. En una notebook realice lo siguiente:

1.Importa las librerías básicas y especializadas que consideres necesarias

In [173]:
## Librerias básicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

!pip install lifelines

## Librerias especializadas
from lifelines import CoxPHFitter
import seaborn as sns
from lifelines import ExponentialFitter, WeibullFitter, LogNormalFitter
from lifelines import LogNormalAFTFitter

Importa la base de datos https://raw.githubusercontent.com/jimmyzac/Estadistica-Aplicada-FCFM-UANL/main/bases_datos/desercion_laboral.csv

In [174]:
datos = pd.read_csv('https://raw.githubusercontent.com/jimmyzac/Estadistica-Aplicada-FCFM-UANL/main/bases_datos/desercion_laboral.csv')

In [175]:
datos

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,departamento,salario
0,3.8,5.3,2,157,3,0,1,0,ventas,bajo
1,8.0,8.6,5,262,6,0,1,0,ventas,medio
2,1.1,8.8,7,272,4,0,1,0,ventas,medio
3,7.2,8.7,5,223,5,0,1,0,ventas,bajo
4,3.7,5.2,2,159,3,0,1,0,ventas,bajo
...,...,...,...,...,...,...,...,...,...,...
14994,4.0,5.7,2,151,3,0,1,0,soporte,bajo
14995,3.7,4.8,2,160,3,0,1,0,soporte,bajo
14996,3.7,5.3,2,143,3,0,1,0,soporte,bajo
14997,1.1,9.6,6,280,4,0,1,0,soporte,bajo


Depurar la base de datos y estadísticas descriptivas

a)Verificar que no haya missing values, en caso de haberlos decidir imputar o eliminar

In [176]:
datos.isnull().sum()

satisfaccion           0
evaluacion             0
proyectos              0
promedio_horas_mes     0
semestre               0
accidente              0
renunciar              0
ascenso_5years         0
departamento          26
salario                0
dtype: int64

In [177]:
#Eliminar missing values
datos = datos.dropna()

In [178]:
datos.isnull().sum()

satisfaccion          0
evaluacion            0
proyectos             0
promedio_horas_mes    0
semestre              0
accidente             0
renunciar             0
ascenso_5years        0
departamento          0
salario               0
dtype: int64

b)Verificar que no haya valores duplicados con dataframe.duplicated().sum()en caso de haber valores duplicados eliminarlos con dataframe=dataframe.drop_duplicates(keep='first').reset_index(drop=True)

In [179]:
datos.duplicated().sum()

2982

In [180]:
# eliminar datos duplicados
datos = datos.drop_duplicates(keep='first').reset_index(drop=True)

In [181]:
datos

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,departamento,salario
0,3.8,5.3,2,157,3,0,1,0,ventas,bajo
1,8.0,8.6,5,262,6,0,1,0,ventas,medio
2,1.1,8.8,7,272,4,0,1,0,ventas,medio
3,7.2,8.7,5,223,5,0,1,0,ventas,bajo
4,3.7,5.2,2,159,3,0,1,0,ventas,bajo
...,...,...,...,...,...,...,...,...,...,...
11986,3.7,4.7,2,152,3,0,1,0,desarrollo de producto,bajo
11987,8.4,9.9,4,267,5,0,1,0,desarrollo de producto,bajo
11988,4.1,4.6,2,151,3,0,1,0,desarrollo de producto,bajo
11989,7.6,9.2,4,239,5,0,1,0,desarrollo de producto,bajo


c)Verificar  que  todas  las  variables  sean  numéricas,  si  no  son  numéricas hacerlas   numéricas,   en   el   caso   de   las   categóricas   convertirlas   en dummies y conservar n-1 dummies

In [182]:
datos.dtypes

satisfaccion          float64
evaluacion            float64
proyectos               int64
promedio_horas_mes      int64
semestre                int64
accidente               int64
renunciar               int64
ascenso_5years          int64
departamento           object
salario                object
dtype: object

In [183]:
datos.value_counts('departamento')

departamento
ventas                    3239
mantenimiento             2244
soporte                   1821
informática                976
Investigación              694
desarrollo de producto     686
mercadotecnia              673
contabilidad               621
recursos humanos           601
administración             436
dtype: int64

In [184]:
## cambiar los valores de texto (departamento), a valores enteros
condiciones = [datos['departamento']=="ventas",
               datos['departamento']=="mantenimiento",
               datos['departamento']=="soporte",
               datos['departamento']=="informática",
               datos['departamento']=="desarrollo de producto",
               datos['departamento']=="mercadotecnia",
               datos['departamento']=="Investigación",
               datos['departamento']=="contabilidad",
               datos['departamento']=="recursos humanos",
               datos['departamento']=="administración"]
valor_condicion = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
datos['departamento'] = np.select(condiciones, valor_condicion, default='Invalid')

In [185]:
# cambiar los valores de texto (salario), a valores enteros
condicion = [datos['salario']=="alto",
             datos['salario']=="medio",
             datos['salario']=="bajo"]
valor = [0, 1, 2]
datos['salario'] = np.select(condicion, valor, default='Invalid')

In [186]:
datos

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,departamento,salario
0,3.8,5.3,2,157,3,0,1,0,0,2
1,8.0,8.6,5,262,6,0,1,0,0,1
2,1.1,8.8,7,272,4,0,1,0,0,1
3,7.2,8.7,5,223,5,0,1,0,0,2
4,3.7,5.2,2,159,3,0,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...
11986,3.7,4.7,2,152,3,0,1,0,4,2
11987,8.4,9.9,4,267,5,0,1,0,4,2
11988,4.1,4.6,2,151,3,0,1,0,4,2
11989,7.6,9.2,4,239,5,0,1,0,4,2


In [187]:
#dummies
dummie_dep = pd.get_dummies(datos['departamento'], prefix='dep').astype(int)
dummie_sal = pd.get_dummies(datos['salario'], prefix='sal').astype(int)

In [188]:
#dummies del departamento
dummie_dep.value_counts('dep_9')

dep_9
0    11555
1      436
dtype: int64

In [189]:
dummie_dep

,dep_0,dep_1,dep_2,dep_3,dep_4,dep_5,dep_6,dep_7,dep_8,dep_9
0,1,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
11986,0,0,0,0,1,0,0,0,0,0
11987,0,0,0,0,1,0,0,0,0,0
11988,0,0,0,0,1,0,0,0,0,0
11989,0,0,0,0,1,0,0,0,0,0


In [190]:
dummie_dep.describe()

,dep_0,dep_1,dep_2,dep_3,dep_4,dep_5,dep_6,dep_7,dep_8,dep_9
count,11991.000000,11991.00000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000
mean,0.270119,0.18714,0.151864,0.081394,0.057210,0.056125,0.057877,0.051789,0.050121,0.036361
std,0.444040,0.39004,0.358904,0.273451,0.232252,0.230173,0.233520,0.221610,0.218204,0.187194
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [191]:
dummie_dep = dummie_dep[['dep_0', 'dep_1', 'dep_2', 'dep_3', 'dep_4', 'dep_5', 'dep_6', 'dep_7', 'dep_8']]

In [192]:
datos = pd.concat([datos, dummie_dep], axis=1)

In [193]:
#dummies del salario
dummie_sal.value_counts('sal_2')

sal_2
0    6251
1    5740
dtype: int64

In [194]:
datos = datos.drop('departamento', axis=1)

In [195]:
dummie_sal

,sal_0,sal_1,sal_2
0,0,0,1
1,0,1,0
2,0,1,0
3,0,0,1
4,0,0,1
...,...,...,...
11986,0,0,1
11987,0,0,1
11988,0,0,1
11989,0,0,1


In [196]:
dummie_sal.describe()

,sal_0,sal_1,sal_2
count,11991.000000,11991.000000,11991.000000
mean,0.082562,0.438746,0.478692
std,0.275230,0.496254,0.499567
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000


In [197]:
dummie_sal = dummie_sal[['sal_1', 'sal_2']]

In [198]:
datos = pd.concat([datos, dummie_sal], axis=1)

In [199]:
datos = datos.drop('salario', axis=1)

In [200]:
datos.dtypes

satisfaccion          float64
evaluacion            float64
proyectos               int64
promedio_horas_mes      int64
semestre                int64
accidente               int64
renunciar               int64
ascenso_5years          int64
dep_0                   int64
dep_1                   int64
dep_2                   int64
dep_3                   int64
dep_4                   int64
dep_5                   int64
dep_6                   int64
dep_7                   int64
dep_8                   int64
sal_1                   int64
sal_2                   int64
dtype: object

In [201]:
# verificar si existen missing values
datos.isnull().sum()

satisfaccion          0
evaluacion            0
proyectos             0
promedio_horas_mes    0
semestre              0
accidente             0
renunciar             0
ascenso_5years        0
dep_0                 0
dep_1                 0
dep_2                 0
dep_3                 0
dep_4                 0
dep_5                 0
dep_6                 0
dep_7                 0
dep_8                 0
sal_1                 0
sal_2                 0
dtype: int64

d)Calcular estadísticas descriptivas

In [202]:
datos.describe()

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,dep_0,dep_1,dep_2,dep_3,dep_4,dep_5,dep_6,dep_7,dep_8,sal_1,sal_2
count,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.00000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000
mean,6.296581,7.166825,3.802852,200.473522,3.364857,0.154282,0.166041,0.016929,0.270119,0.18714,0.151864,0.081394,0.057210,0.056125,0.057877,0.051789,0.050121,0.438746,0.478692
std,2.410700,1.683426,1.163238,48.727813,1.330240,0.361234,0.372133,0.129012,0.444040,0.39004,0.358904,0.273451,0.232252,0.230173,0.233520,0.221610,0.218204,0.496254,0.499567
min,0.900000,3.600000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.800000,5.700000,3.000000,157.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.600000,7.200000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.200000,8.600000,5.000000,243.000000,4.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,10.000000,10.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


4.Estimar el modelode Cox e interpretar cada uno de los coeficientes

In [203]:
cphf = CoxPHFitter().fit(datos, 'semestre', 'renunciar')
cphf.print_summary()

<lifelines.CoxPHFitter: fitted with 11991 total observations, 10000 right-censored observations>
             duration col = 'semestre'
                event col = 'renunciar'
      baseline estimation = breslow
   number of observations = 11991
number of events observed = 1991
   partial log-likelihood = -15884.05
         time fit was run = 2023-09-27 04:57:41 UTC

---
                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                  
satisfaccion        -0.24       0.78       0.01            -0.26            -0.22                 0.77                 0.80
evaluacion          -0.01       0.99       0.01            -0.04             0.02                 0.96                 1.02
proyectos           -0.29       0.75       0.02            -0.33            -0.25                 0.72                 0.78
promedio_horas_mes   0.00       1.00       0.00             0.00             0.00                 1.00                 1.00
accidente           -1.25       0.29       0.10            -1.44            -1.05                 0.24                 0.35
ascenso_5years      -1.38       0.25       0.36            -2.08            -0.68                 0.13                 0.51
dep_0                0.13       1.14       0.15            -0.16             0.42                 0.85                 1.52
dep_1                0.21       1.24       0.15            -0.08             0.50                 0.92                 1.66
dep_2                0.23       1.26       0.15            -0.07             0.53                 0.94                 1.69
dep_3                0.07       1.07       0.16            -0.24             0.39                 0.78                 1.47
dep_4                0.14       1.15       0.17            -0.19             0.47                 0.82                 1.60
dep_5                0.12       1.13       0.17            -0.21             0.45                 0.81                 1.57
dep_6               -0.22       0.80       0.18            -0.57             0.13                 0.57                 1.14
dep_7                0.02       1.02       0.17            -0.31             0.35                 0.73                 1.42
dep_8                0.21       1.24       0.17            -0.12             0.55                 0.89                 1.72
sal_1                1.15       3.15       0.15             0.85             1.44                 2.35                 4.22
sal_2                1.54       4.66       0.15             1.25             1.83                 3.48                 6.23

                     cmp to      z      p   -log2(p)
covariate                                           
satisfaccion           0.00 -26.89 <0.005     526.58
evaluacion             0.00  -0.76   0.44       1.17
proyectos              0.00 -14.75 <0.005     161.15
promedio_horas_mes     0.00   3.88 <0.005      13.25
accidente              0.00 -12.41 <0.005     115.04
ascenso_5years         0.00  -3.88 <0.005      13.21
dep_0                  0.00   0.89   0.37       1.42
dep_1                  0.00   1.42   0.16       2.68
dep_2                  0.00   1.53   0.13       2.98
dep_3                  0.00   0.45   0.66       0.61
dep_4                  0.00   0.82   0.41       1.29
dep_5                  0.00   0.70   0.48       1.05
dep_6                  0.00  -1.24   0.21       2.22
dep_7                  0.00   0.12   0.91       0.14
dep_8                  0.00   1.27   0.21       2.29
sal_1                  0.00   7.67 <0.005      45.68
sal_2                  0.00  10.36 <0.005      81.15
---
Concordance = 0.85
Partial AIC = 31802.11
log-likelihood ratio test = 1360.94 on 17 df
-log2(p) of ll-ratio test = 924.90

In [204]:
cphf.params_

covariate
satisfaccion         -0.242337
evaluacion           -0.010687
proyectos            -0.291606
promedio_horas_mes    0.001839
accidente            -1.246686
ascenso_5years       -1.380084
dep_0                 0.130262
dep_1                 0.211686
dep_2                 0.230800
dep_3                 0.071793
dep_4                 0.139730
dep_5                 0.118564
dep_6                -0.220285
dep_7                 0.020104
dep_8                 0.214053
sal_1                 1.147612
sal_2                 1.538901
Name: coef, dtype: float64

In [205]:
cphf.hazard_ratios_

covariate
satisfaccion          0.784792
evaluacion            0.989370
proyectos             0.747063
promedio_horas_mes    1.001841
accidente             0.287456
ascenso_5years        0.251557
dep_0                 1.139127
dep_1                 1.235759
dep_2                 1.259607
dep_3                 1.074433
dep_4                 1.149964
dep_5                 1.125879
dep_6                 0.802290
dep_7                 1.020307
dep_8                 1.238688
sal_1                 3.150662
sal_2                 4.659468
Name: exp(coef), dtype: float64

Considerando la hipótesis nula

$H_0: \beta = 0$

contra la hipótesis alternativa

$(H_a: \beta \neq 0)$

con un nivel de significancia

 $(\alpha = 0.05)$, podemos usar la regla general:

1. **satisfaccion**: El coeficiente es -0.24. El p-valor es prácticamente cero (<0.005), lo que significa que hay evidencia suficiente para rechazar la hipótesis nula. Por lo tanto, el riesgo de renunciar disminuye en un 22% por cada unidad de aumento en satisfacción.

2. **evaluacion**: El coeficiente es -0.01. El p-valor es 0.44, lo que es mayor que $(\alpha)$. No hay suficiente evidencia para rechazar la hipótesis nula, por lo que la evaluación no parece estar significativamente asociada con el tiempo hasta la renuncia.

3. **proyectos**: El coeficiente es -0.29. El p-valor es prácticamente cero (<0.005), lo que significa que hay evidencia suficiente para rechazar la hipótesis nula. Por lo tanto, el riesgo de renunciar disminuye en un 25% por cada unidad de aumento en el número de proyectos.

4. **promedio_horas_mes**: El coeficiente es muy cercano a cero (0.00) y el p-valor es pequeño (<0.005). No hay cambio en el riesgo de renuncia por cada unidad de aumento en el promedio de horas trabajadas por mes.

5. **accidente**: El coeficiente es -1.25. El p-valor es prácticamente cero (<0.005), lo que significa que hay evidencia suficiente para rechazar la hipótesis nula. Por lo tanto, los empleados que tuvieron un accidente tienen un 71% menos de riesgo de renunciar en comparación con los que no lo tuvieron.

6. **ascenso_5years**: El coeficiente es -1.38. El p-valor es prácticamente cero (<0.005), lo que significa que hay evidencia suficiente para rechazar la hipótesis nula. Por lo tanto, los empleados que no tuvieron ascenso en los últimos 5 años tienen un 75% menos de riesgo de renunciar en comparación con los que sí lo tuvieron.

7. **dep_X** (para los valores de dep_0 a dep_9): Todos los coeficientes tienen p-valores mayores al alfa, lo que significa que no hay suficiente evidencia para rechazar la hipótesis nula, esto indica que ninguna de las variables de departamento (dep_X) tiene un impacto significativo en el tiempo hasta la renuncia, en comparación con el departamento de referencia.

8. **sal_1** : El coeficiente es 1.15. El p-valor es prácticamente cero (<0.005), lo que significa que hay evidencia suficiente para rechazar la hipótesis nula. Por lo tanto, los empleados en la categoría sal_1 tienen un 215% más de riesgo de renuncia en comparación con los del grupo de referencia.

9. **sal_2** : El coeficiente es 1.54. El p-valor es prácticamente cero (<0.005), lo que significa que hay evidencia suficiente para rechazar la hipótesis nula. Por lo tanto, los empleados en la categoría sal_2 tienen un 366% más de riesgo de renuncia en comparación con los del grupo de referencia.